In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from global_variables import DIM

from simulation import read_results, read_params
from global_variables import DIM
from plotting_tools import add_plot_decoration, generate_labels
from matplotlib.colors import LogNorm
from plotting_tools import plot_noise

%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Noise analysis

In order to analyse algorythms robustness to noise we have to know the scale of measrurements. 
We can see on the plot below, that in the current setup, the distances have a multimodal distribution that is looks like a mixture of gaussians (with the difference that it's only positive, or that the bigger gaussian is squed). The distance wary between $0.1$ and $25$.

For analysis of real distances, see below.

In [ ]:
key = 'get_distances'
save_figures = True

resultfolder = 'results/{}/'.format(key)
results = read_results(resultfolder + 'result_')
#parameters = read_params(resultfolder + 'parameters.json')

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5

ist, bins, _ = plt.hist(np.sqrt(results['distances']), bins=100)
if save_figures:
    plt.savefig(resultfolder + "simulated_distances.pdf", bbox_inches="tight")
plt.title("simulated distance distribution")
plt.show()
logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
plt.hist(np.sqrt(results['distances']), bins=logbins)
plt.xscale('log')
plt.title("simulated distance distribution (log scale)")
if save_figures:
    plt.savefig(resultfolder + "simulated_distances_log.pdf", bbox_inches="tight")
plt.show()

## Real distance distribution

In [ ]:
from evaluate_dataset import read_anchors_df, read_dataset

names =  ['circle2_double.csv', 
             'circle3_triple.csv', 
             'clover.csv',
             'eight2_double.csv', 
             'rounds.csv', 
             'straight1.csv', 
             'straight2.csv', 
             'straight3.csv', 
             'straight4.csv', 
             'straight5.csv', 
             'straight6.csv', 
             'triangle_double.csv']

anchorsfile = 'experiments/anchors.csv'
anchors_df = read_anchors_df(anchorsfile)
anchors = anchors_df[["px", "py", "pz"]].values.T

noisy_distances = []

for name in names:
    datafile = 'experiments/robot_test/' + name
    data_df = read_dataset(datafile, anchors_df)
    noisy_distances.extend(data_df[data_df.system_id=="RTT"]["distance"].values.tolist())
np.savetxt(resultfolder + 'noisy_distances.csv', np.array(noisy_distances))

In [ ]:
import pickle
from trajectory import Trajectory
from measurements import get_D_topright

MM = 0.001

with open('controls/robot_trajectores.pkl', 'rb') as f:
    trajectories = pickle.load(f)

distances = []
n_samples = 400
    
for traj in trajectories:
    trajectory = Trajectory(**traj['parameters'])
    # Dont strech trajectories that have hand designed coefficients
    trajectory.scale_bounding_box(traj['box'] * MM, keep_aspect_ratio='coeffs' in traj['parameters'])
    trajectory.center()
    basis = trajectory.get_basis(n_samples=n_samples)
    positions = trajectory.get_sampling_points(basis)
    positions = np.concatenate([positions, np.zeros((1, n_samples))])
    distances_squared = get_D_topright(anchors=anchors, samples=positions)
    distances.extend(np.sqrt(distances_squared.flatten()).tolist())

In [ ]:
def plot_pair(dist_dict, save_figures=False):
    """
    Plot distance histograms in normal and log scale. 
    
    :param dist_dict: dictionary with either {label:distances} or {label:filename}, 
                      where distances is the list of simulated distances (see cell above),
                      or filename is the name of the txtfile where data was stored (see 2 cells above).
    
    """
    fig, axs = plt.subplots(2, 1)
    fig.set_size_inches(10, 7)
    
    axs[0].set_title("distance distribution")
    axs[1].set_title("noisy distance distribution (log scale)")
    
    bins = np.linspace(0.5, 20, 100)
    logbins = np.logspace(np.log10(bins[0]),np.log10(bins[-1]),len(bins))
    
    for label, distances in dist_dict.items():
        if type(distances) == str: # distances is just a file name.
            distances = np.loadtxt(resultfolder + distances)
            
        axs[0].hist(distances, bins=bins, label=label, alpha=0.7)
        axs[1].hist(distances, bins=logbins, label=label, alpha=0.7)
        axs[1].set_xscale('log')
        
    if save_figures:
        plt.savefig(resultfolder + "distances.pdf", bbox_inches="tight")
            
    axs[0].legend()
    axs[1].legend()
    return fig

fig = plot_pair({"noisy":"noisy_distances.csv", "simulated":distances})

Above we can see real distance distribution. Note that the true distances are not realy the true distances, but a simulation where the trajectory is in the middle of the room. I think for our pourposes it's enough. Might be good to calibrate the noisy distances before plotting them here (as it's what we will be using for reconstruction).

We can see at the plots below, that the noisy distances (without callibration) range between $1$ and $20$ meters, where $20$ meters is clearly and outlier. The shape of the disriburion seems to be unimodal (which is good, in general), and match the shape of the bigger of the simulated modes. The true distances fit nicely between $1$ and $8$, and are (more or less) unimodal. The log plot looks shifted, which would suggest that noisy distances are multiplied by some factor (again, maybe I should calibrate distances before plotting).

It seems to me that the simulated distribution is more difficult to work with than the real one (which means that if algorytms below work for simulations, they will work for true data). The only issue is that we don't know the noise distribution of the real data.

# Calibrated distance distribution

In below cells we plot calibrated vs. tango vs. original ("noisy") distance measurements. We see that calibration corrects the offset that we saw in above plots. We also see that the Michalina's hack to quickly calculate distances from the trajectory by centering it is reasonably close to the actual distances we get from tango.

In [ ]:
import pandas as pd

calib_distances = []
tango_distances = []

for name in names:
    calib_name = name.replace('.csv', '_calibrated.pkl')
    datafile = 'experiments/robot_test/' + calib_name
    data_df = pd.read_pickle(datafile)
    print('read', datafile)
    calib_distances.extend(data_df[data_df.system_id=="RTT"]["distance_median_0"].values.tolist())
    tango_distances.extend(data_df[data_df.system_id=="RTT"]["distance_tango"].values.tolist())
np.savetxt(resultfolder + 'calib_distances.csv', np.array(calib_distances))
np.savetxt(resultfolder + 'tango_distances.csv', np.array(tango_distances))

In [ ]:
fig = plot_pair({"tango":"tango_distances.csv", 
           "calib":"calib_distances.csv", 
           "noisy":"noisy_distances.csv"})
fig.suptitle("Comparing non-calibrated to calibrated distances.")

In [ ]:
fig = plot_pair({"tango":"tango_distances.csv", 
                 "simulated":distances})
fig.suptitle("Comparing simulated 'real' distances to Tango data.")

## Noise added to distances, basic right inverse
In this case, the noise is added to measurements, what leads to adding `distance x noise` to squared distances (assuming that we can neglect $\sigma^2$). If we use a version of OLS (right inverse), we assume the noise to be gaussian. This might lead to bigger errors. 

Below, we can see reconstruction errors for different number of available measurements and different noise $\sigma$s. We consider noise up to $\sigma = 10$, because that is still within the range of distance measurements. 

We plot three different errors - relative and absolute distance errors, and coefficient reconstruction errors. The distance errors is what is more intuitive. We should think what metric we should report. 

We can see that there is huge difference on betwen absolute and relative errors, which suggest large errors in reconstruction of large distances. Or maybe it's just because we take a mean and not meadian error. 

We can see that with oversampling the error drops more or less linearly on the log-log plot, with $<10\times$ oversampling leads to more than $10\times$ decrease in error. However, some of this decrease (see the nonlinear behaviour of the plots for small number of samples) comes from the rank deficiency, which is more likely for small number of samples. We can see roguhly the same drop on all plots.

In [ ]:
rcParams['figure.figsize'] = 10, 5
plot_noise('noise_right_inverse', save_figures=True, max_noise=-1)

## Noise added to distances, weighted right inverse
In this case, the noise is added to measurements, what leads to adding `distance x noise` to squared distances (assuming we can neglect $\sigma^2$). We can try to normalize the error, by dividing by the (noisy) distance. I divide by the noisy distance $+10^{-3}$, to avoid dividing by really small distances.

The results are more or less the same as for OLS, but it seems that the improvement with oversampling is sligthly bigger, with better errors on coefficients. 

In [ ]:
plot_noise('noise_right_inverse_weighted', save_figures=True,  max_noise=-1)

## Noise added to squared distances
In this case the model matches OLS assumptions, so there is no need to use WLS. It's mostly for reference, to see if there is any strange behaviour that comes from trajectory model and not the noise model.

We can see some outliers for large noise, but that could go away with more simulations. Overall, the behaviour is similar than the previous plots, with maybe $10\times$ smaller relative and coefficient errors. Seems that the squared noise is not a big issue. 

In [ ]:
plot_noise('noise_to_square_right_inverse', save_figures=True,  max_noise=-1)